In [17]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_california_housing
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import SGD
import torch.utils.data as Data
import matplotlib.pyplot as plt
import seaborn as sns
minloss=float('inf')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


#导入数据
housedata=fetch_california_housing()
#划分测试集和训练集
train_x,test_x,train_y,test_y=train_test_split(housedata.data,housedata.target,test_size=0.3,random_state=42)
#标准化处理
scale=StandardScaler()
train_x=scale.fit_transform(train_x)
test_x=scale.fit_transform(test_x)



#将数据集转化为张量 并处理为PyTorch网络使用的数据
train_xt=torch.from_numpy(train_x.astype(np.float32))
train_yt=torch.from_numpy(train_y.astype(np.float32))
test_xt=torch.from_numpy(test_x.astype(np.float32))
test_yt=torch.from_numpy(test_y.astype(np.float32))
#将数据处理为数据加载器
train_data=Data.TensorDataset(train_xt,train_yt)
test_data=Data.TensorDataset(test_xt,test_yt)
train_loader=Data.DataLoader(dataset=train_data,batch_size=64,shuffle=True,num_workers=0)

# neural network model mlp
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.hidden1 = nn.Sequential(nn.Linear(in_features=8, out_features=256, bias=True),
                                  nn.Dropout(0.7),
                                  nn.ReLU())
        
        self.hidden2 = nn.Sequential(nn.Linear(in_features=256, out_features=64, bias=True),
                                  nn.Dropout(0.7),
                                  nn.ReLU())
        
        self.hidden3 = nn.Sequential(nn.Linear(in_features=64, out_features=16, bias=True),
                                  nn.Dropout(0.5),
                                  nn.ReLU()
                                  )
        
        self.predict = nn.Sequential(nn.Linear(in_features=16, out_features=1, bias=True),
                                        nn.ReLU())
        
    def forward(self, x):
        x = self.hidden1(x)
        x = self.hidden2(x)
        x = self.hidden3(x)
        output = self.predict(x)
        return output[:, 0]


mlpreg=MLP().to(device)
print(mlpreg)


#定义优化器
optimizer=torch.optim.Adam(mlpreg.parameters(),lr=5e-5)
loss_func=nn.MSELoss()
train_loss_all=[]
for epoch in range(6000):
    train_loss=0
    train_num=0
    for step,(b_x,b_y) in enumerate(train_loader):
        output=mlpreg(b_x.to(device))
        loss=loss_func(output.to(device),b_y.to(device))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss+=loss.item()*b_x.size(0)
        train_num+=b_x.size(0)
    train_loss_all.append(train_loss/train_num)
    if epoch%10 == 0:
        print(f'train: Epo:{epoch}  Loss_stu:{loss}')
plt.figure(figsize=(10,6))
plt.plot(train_loss_all,"ro-",label="Train loss")
plt.legend()
plt.grid()
plt.xlabel("epoch")
plt.ylabel("loss")
plt.show()



MLP(
  (hidden1): Sequential(
    (0): Linear(in_features=8, out_features=256, bias=True)
    (1): Dropout(p=0.7, inplace=False)
    (2): ReLU()
  )
  (hidden2): Sequential(
    (0): Linear(in_features=256, out_features=64, bias=True)
    (1): Dropout(p=0.7, inplace=False)
    (2): ReLU()
  )
  (hidden3): Sequential(
    (0): Linear(in_features=64, out_features=16, bias=True)
    (1): Dropout(p=0.5, inplace=False)
    (2): ReLU()
  )
  (predict): Sequential(
    (0): Linear(in_features=16, out_features=1, bias=True)
    (1): ReLU()
  )
)
train: Epo:0  Loss_stu:2.8673319816589355


In [ ]:
#预测


#预测
pre_y=mlpreg(test_xt.to(device))
pre_y=pre_y.data.cpu().numpy()
print(test_y,pre_y)
 
mae=mean_squared_error(test_y,pre_y)
print(mae)
index=np.argsort(test_yt)
plt.figure(figsize=(12,5))
plt.plot(np.arange(len(test_y)),test_y[index],"r",label="original y")
plt.scatter(np.arange(len(pre_y)),pre_y[index],s=3,c="b",label="prediction")
plt.legend(loc="upper left")
plt.grid()
plt.xlabel("index")
plt.ylabel("y")
plt.show()
#1.3248